In [5]:
# TODO sampling logic (undersample)
# TODO binary qa prompt and co need to vary between a few samples
# TODO determine hyperparams
# TODO arg parsing
# TODO dataloader num workers set to default

# SOME NOTES
# PPO_TRAINER AUTOMATICALLY PADS THE INPUTS BY TOKENIZER.PADDING_SIDE AND TOKENIZER.PADDING_TOKEN_ID
# Uh-oh, because ppo termination token is set as the eos_seq_token, it'll stop when it sees a left padded sequence
# Skipping random exploration for now


# BATCH TIMING
# A batch of 8 samples take around 1-1.5-2-3min to process in a train step (so around 400 samples per hour is trainable, every 50th batch, we save a checkpoint, and do val)
# Lets save a checkpoint every half an hour or so
# Give validation around 15 mins => 100 samples or so
# Validation is around 8k so it'll be 1000 batches (1000*1.5 min = 25 hours)
# len(dataset_eval) = 8737

In [1]:
# %% Set script for interactive development and import modules
from RewardingVisualDoubt import infrastructure, training

infrastructure.make_ipython_reactive_to_changing_codebase()
infrastructure.supress_known_warnings()

import pathlib as path
import typing as t
import torch
import numpy as np
import time
import os
from torch.utils.data import DataLoader
import accelerate
import dataclasses
import functools

# from LLAVA_Biovil.llava.mm_utils import KeywordsStoppingCriteria
from trl import PPOConfig, PPOTrainer
import trl

from RewardingVisualDoubt import dataset, prompter, shared, vllm, response, reward
from RewardingVisualDoubt import training as training

os.environ["WANDB_API_KEY"] = "da3cb086bbc110c16cbc5ba4c284a19b0b461710"

from LLAVA_Biovil.llava.mm_utils import KeywordsStoppingCriteria

STOP_STR = prompter.Seperator.END_OF_SEQUENCE_SEPERATOR.value

Fetching 69 files:   0%|          | 0/69 [00:00<?, ?it/s]

Fetching 69 files:   0%|          | 0/69 [00:00<?, ?it/s]

In [2]:
model = vllm.load_pretrained_llava_model_without_vision_support_and_without_lora_adapters(
    device=shared.torch_devices.cuda.value,
    precision="16bit",
)

Loading the model in without vision support and withour LoRA adapters, freezing all layers...
Precision: 16bit (non-quantized)
Model base:  liuhaotian/llava-v1.5-7b
Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading additional LLaVA weights...


In [3]:
model = vllm.add_pretrained_RaDialog_lora_adapters_to_LlavaLlamaForCausalLM_model(
    model,
    radialog_lora_weights_path=vllm.RadialogLoraWeightsPath.BINARY_QA_WITH_CONFIDENCE_SFT.value,
)

Adding pretrained RaDialog LoRA adapters to the model...


In [5]:
model = model.merge_and_unload()

In [6]:
model

LlavaLlamaForCausalLM(
  (model): LlavaLlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): 

In [7]:
path = os.path.join(
    vllm.MODEL_SAVING_DIR, "radialog_binary_qa_with_confidence_sft_full_merged_model"
)

In [9]:
model.save_pretrained(path)

In [2]:
############################################ For prototyping only: Input hyperparameters ########################################
NUM_EPOCHS = 1
DEFAULT_BATCH_SIZE = 8
GRADIENT_ACCUMULATION_STEPS = 2
MINI_BATCH_SIZE = int(DEFAULT_BATCH_SIZE / 2)
LEARNING_RATE = 5e-5
DEFAULT_OUTPUT_DIR = path.Path("output")

batch_size = DEFAULT_BATCH_SIZE
num_epochs = NUM_EPOCHS
batch_size = DEFAULT_BATCH_SIZE
gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS
mini_batch_size = int(DEFAULT_BATCH_SIZE / 2)
learning_rate = LEARNING_RATE
out_dir: path.Path = DEFAULT_OUTPUT_DIR

######################################## 0. Define the environment ########################################

device_str = (
    shared.torch_devices.cuda.value if torch.cuda.is_available() else shared.torch_devices.cpu.value
)
device = torch.device(device_str)

######################################## 1. Load the model and tokenizer ########################################

model = vllm.load_pretrained_llava_model_for_ppo_training(
    device_str=device_str,
    precision="4bit",
    radialog_lora_weights_path=vllm.RadialogLoraWeightsPath.BINARY_QA_WITH_CONFIDENCE_SFT.value,
)

tokenizer = vllm.load_pretrained_llava_tokenizer_with_image_support(
    model_base=vllm.LLAVA_BASE_MODEL_NAME
)
padding_tokenizer = vllm.load_pretrained_llava_tokenizer_with_image_support(
    model_base=vllm.LLAVA_BASE_MODEL_NAME
)
padding_tokenizer.padding_side = "left"


######################################## 2. Load the datasets and the dataloaders ########################################

print("Loading the datasets and the dataloaders...")
prompter_ = functools.partial(
    prompter.build_binary_qa_prompt_with_response_and_confidence_for_sft, is_for_inference=True
)
dataset_train = dataset.get_binary_qa_prompted_mimic_cxr_llava_model_input_dataset(
    split=dataset.DatasetSplit.TRAIN,
    tokenizer=tokenizer,
    prompter=prompter_,
)
dataset_eval = dataset.get_binary_qa_prompted_mimic_cxr_llava_model_input_dataset(
    split=dataset.DatasetSplit.VALIDATION,
    tokenizer=tokenizer,
    prompter=prompter_,
)

dataloader_train = dataset.get_mimic_cxr_llava_model_input_dataloader(
    dataset=dataset_train,
    batch_size=batch_size,
    padding_tokenizer=padding_tokenizer,
    num_workers=8,
)

dataloader_eval = dataset.get_mimic_cxr_llava_model_input_dataloader(
    dataset=dataset_eval,
    batch_size=2 * batch_size,
    padding_tokenizer=padding_tokenizer,
    num_workers=8,
)

eval_batch_iterator = iter(dataloader_eval)

import sys

sys.path.append("../..")  # Adds higher directory to python modules path.
from workflows import radialog_binary_qa_ppo_training

Adding LoRA adapters and value head to the model for PPO training using Radialog Lora Weights path: /home/guests/deniz_gueler/repos/RewardingVisualDoubt/workflows/training_checkpoints/best_model_epoch0_step179.pth/adapter_model.bin
Loading the model in non-trainable mode...
Precision: 4bit quantized
Model base:  liuhaotian/llava-v1.5-7b
Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading additional LLaVA weights...
Using downloaded and verified file: /tmp/biovil_t_image_model_proj_size_128.pt


Loaded additional vision tower weights...
Adding pretrained RaDialog LoRA adapters and value head to the model...


/home/guests/deniz_gueler/miniconda3/envs/llava_hf/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Loading the datasets and the dataloaders...
Loading mimic_cxr_df from cache
Loading balanced_binary_qa_mimic_cxr_df from cache
Loading mimic_cxr_df from cache
Loading balanced_binary_qa_mimic_cxr_df from cache
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
######################################## 3. Define the PPO and generation configurations ########################################

# Example of batch size 16: 4 epochs over the batch. Each backward batch is of size 8, and each mini batch is of size 4
# Gradients get accumulated during 4 + 4 mini batches, and then the model gets updated (the "backward batch" is completed)

ppo_config = trl.PPOConfig(
    learning_rate=learning_rate,
    task_name="gpt",
    ppo_epochs=1,  # Default value from TRL library is 4 (i.e. will go over the batch 4 times), but since we have a lot of data, we can set it to 1
    batch_size=batch_size,
    # backward_batch_size=MINI_BATCH_SIZE,  # Default value from TRL library is 1, gets overwritten anyways at __init__ time
    mini_batch_size=mini_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    log_with="wandb",
    tracker_project_name="radialog_binary_qa_ppo_training",
    project_kwargs=dataclasses.asdict(
        accelerate.utils.ProjectConfiguration(
            project_dir="radialog_binary_qa_ppo_training", logging_dir="logs"
        )
    ),
    remove_unused_columns=False,
    # optimize_device_cache=True,
    kl_penalty="kl",  # 'kl': model_logp - ref_logp,  'abs': abs(kl),  'mse': mean squared error mse(kl) and 'full': the actual kl for all tokens in the distribution"
    init_kl_coef=0.05,
)

generation_kwargs_ppo = {
    "min_length": -1,  # don't ignore the EOS token (see above)
    "top_k": 0.0,  # no top-k sampling
    "top_p": 1.0,  # no nucleus sampling
    "temperature": 1.0,  # DONT BE CREATIVE
    "do_sample": True,  # yes, we want to sample
    "pad_token_id": tokenizer.pad_token_id,  # most decoder models don't have a padding token - use EOS token instead (for this tokenizer it was already set to eos_token_id)
    "max_new_tokens": 50,  # let's not be chatty, we need a few tokens to generate confidence but also let us not limit the response too much
    "eos_token_id": tokenizer.eos_token_id,  # (instead of ppo_terminators list)
}

ppo_trainer = t.cast(
    training.MultimodalPPOTrainer,
    training.MultimodalPPOTrainer(
        model=model,
        config=ppo_config,
        tokenizer=tokenizer,
    ),
)

# not sure if needed but just to be safe for now
tokenizer.padding_side = "left"
model.config.padding_side = "left"
model.config.tokenizer_padding_side = "left"
# model.pad_token_id = tokenizer.eos_token_id

fatal: No names found, cannot describe anything.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: onurdenizguler (onurdenizguler-technical-university-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/guests/deniz_gueler/miniconda3/envs/llava_hf/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:238: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [4]:
rewards_epoch = []
iterator_train = iter(dataloader_train)
batch = next(iterator_train)

# for i in range(1):
#     print(i)
#     batch = next(iterator_train)
#     rewards, batch_report = radialog_binary_qa_ppo_training.radialog_binary_qa_ppo_training_step(
#         model,
#         device,
#         tokenizer,
#         generation_kwargs_ppo,
#         ppo_trainer,
#         batch,
#     )

In [15]:
model.pretrained_model.enable_input_require_grads()

In [30]:
for _ in range(20):
    batch = next(iterator_train)
    radialog_binary_qa_ppo_training.radialog_binary_qa_ppo_training_step(
        model,
        device,
        tokenizer,
        generation_kwargs_ppo,
        ppo_trainer,
        batch,
    )

First pass for logprobs


/home/guests/deniz_gueler/miniconda3/envs/llava_hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Compute rewards and advantages
Start epoch level training
Start backward batch level training for batch size: 8
Start mini batch level training for batch size: 4
Start mini batch level training for batch size: 4
First pass for logprobs
Compute rewards and advantages
Start epoch level training
Start backward batch level training for batch size: 8
Start mini batch level training for batch size: 4
Start mini batch level training for batch size: 4
First pass for logprobs
Compute rewards and advantages
Start epoch level training
Start backward batch level training for batch size: 8
Start mini batch level training for batch size: 4
Start mini batch level training for batch size: 4
First pass for logprobs
Compute rewards and advantages
Start epoch level training
Start backward batch level training for batch size: 8
Start mini batch level training for batch size: 4
Start mini batch level training for batch size: 4
First pass for logprobs
Compute rewards and advantages
Start epoch level trainin

In [16]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

pretrained_model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.Size([128, 4096])
pretrained_model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.Size([4096, 128])
pretrained_model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight torch.Size([128, 4096])
pretrained_model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight torch.Size([4096, 128])
pretrained_model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.Size([128, 4096])
pretrained_model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.Size([4096, 128])
pretrained_model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight torch.Size([128, 4096])
pretrained_model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight torch.Size([4096, 128])
pretrained_model.base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight torch.Size([128, 40

In [ ]:
######### 5.1 Unpack the batch #########
batch: dataset.MimicCxrLlavaModelInputBatchDict = batch
batch_llava_model_input_dict = batch["batch_llava_model_input_dict"]
batch_llava_model_input_dict = dataset.move_llava_model_input_dict_to_device(
    batch_llava_model_input_dict, device
)
input_ids, images = (
    batch_llava_model_input_dict["text_prompt_input_ids"],
    batch_llava_model_input_dict["images"],
)
attention_mask = batch["batch_attention_mask"].to(device)
labels = t.cast(torch.Tensor, batch["batch_labels"]).to(device)
stopping_criteria = KeywordsStoppingCriteria([STOP_STR], tokenizer, input_ids)
input_ids_list = training.remove_preciding_padding_from_batch_tensor(input_ids)

######### 5.2 Generate the binary q&a answer and remove trailing padding tokens #########
model.eval()
model.gradient_checkpointing_disable()
generated_ids = ppo_trainer.generate(
    query_tensor=input_ids_list,  # ppo_trainer.generate() method admits list of tensors, handles padding and batching itself
    images=images,
    return_prompt=False,
    batch_size=input_ids.shape[0],
    use_cache=True,  # => not compatible with gradient checkpointing!
    stopping_criteria=[stopping_criteria],
    **generation_kwargs_ppo,
)

In [ ]:
######### 5.3 Parse the responses and compute the scores #########
generated_texts = tokenizer.batch_decode(generated_ids)
generated_answer_labels = response.parse_binary_labels(generated_texts)
generated_confidence_values = response.parse_confidences(generated_texts)

scores = [
    reward.generated_answer_and_confidence_to_reward(
        generated_answer_label, generated_confidence_value, ground_truth_label
    )
    for generated_answer_label, generated_confidence_value, ground_truth_label in zip(
        generated_answer_labels, generated_confidence_values, labels.bool().tolist()
    )
]

scores = t.cast(
    list[torch.FloatTensor],
    [torch.tensor(s).to(device) for s in scores],
)

In [ ]:
######### 5.7 Take a PPO optimization step #########
model.train()
model.gradient_checkpointing_enable()
stats = ppo_trainer.multimodal_step(
    queries=t.cast(list[torch.LongTensor], input_ids_list),
    responses=t.cast(list[torch.LongTensor], generated_ids),
    scores=scores,
    images=images,
)

First pass for logprobs
Compute rewards and advantages
Start epoch level training
Start backward batch level training for batch size: 8
Start mini batch level training for batch size: 4
Start mini batch level training for batch size: 4


In [9]:
stats

{'objective/kl': array(28.700102, dtype=float32),
 'objective/kl_dist': array([28.699497, 40.355667, 25.354595, 38.656567, 25.70254 , 27.247229,
        24.048939, 19.53579 ], dtype=float32),
 'objective/logprobs': array([[-9.8808277e-01, -5.6742010e-05, -1.8344627e-04, ...,
         -2.8458366e+00, -1.9892011e-02, -1.4901050e-05],
        [-9.7929120e-01, -6.1986910e-05, -1.7486473e-04, ...,
         -2.9000490e+00, -2.8164604e+00, -3.8735516e-04],
        [ 0.0000000e+00,  0.0000000e+00, -3.3997768e+01, ...,
         -1.3208448e+00, -3.2068972e-02, -2.1550717e-04],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -2.3513503e+00, -1.6436458e-01, -1.2146689e-04],
        [-1.0727005e+00, -5.5669188e-05, -1.7557987e-04, ...,
         -2.1816630e+00, -1.0783848e-02, -1.7523613e-05],
        [-4.0202721e+01, -4.0407944e+01, -3.9593071e+01, ...,
         -3.3670032e+00, -6.1285581e-02, -7.4622229e-05]], dtype=float32),
 'objective/ref_logprobs': array([[-

In [8]:
model_inputs = ppo_trainer.prepare_model_inputs(
    queries=t.cast(list[torch.LongTensor], input_ids_list),
    responses=t.cast(list[torch.LongTensor], generated_confidences_ids),
)

model_inputs["images"] = images  # N
model_inputs_names = list(model_inputs.keys())

queries = t.cast(list[torch.LongTensor], input_ids_list)
responses = t.cast(list[torch.LongTensor], generated_confidences_ids)
bs = len(queries)
fbs = ppo_trainer.config.mini_batch_size
all_logprobs = []
all_logits = []
all_masks = []
all_values = []


i = 2
input_kwargs = {key: value[i * fbs : (i + 1) * fbs] for key, value in model_inputs.items()}
# query_batch = queries[i * fbs : (i + 1) * fbs]
# response_batch = responses[i * fbs : (i + 1) * fbs]
with torch.no_grad():
    logits_mini, _, values_mini = model(**input_kwargs)

input_ids_mini = input_kwargs["input_ids"]
attention_mask_mini = input_kwargs["attention_mask"]
images_mini = input_kwargs["images"]

In [12]:
with torch.no_grad():
    all_logprobs, logits_or_none, values, masks = ppo_trainer.batched_forward_pass(
        model,
        queries,
        responses,
        model_inputs,
    )

In [17]:
mini_batch_inds = [0, 1, 2, 3]
mini_batch_dict = {
    "logprobs": all_logprobs[mini_batch_inds],
    "values": values[mini_batch_inds],
    "masks": masks[mini_batch_inds],
    # hacks: the queries and responses are ragged.
    "queries": [queries[i] for i in mini_batch_inds],
    "responses": [responses for i in mini_batch_inds],
}
model_inputs_ = {
    "input_ids": model_inputs["input_ids"][mini_batch_inds],
    "attention_mask": model_inputs["attention_mask"][mini_batch_inds],
    "images": model_inputs["images"][mini_batch_inds],
}

with torch.no_grad():
    logprobs_new, logits_new, vpreds_new, _ = ppo_trainer.batched_forward_pass(
        model,
        mini_batch_dict["queries"],
        mini_batch_dict["responses"],
        model_inputs=model_inputs_,
        return_logits=True,
    )

In [ ]:
logprobs_new.shape[1]

torch.Size([4, 395])

In [22]:
mini_batch_excess_padding_len

0

In [ ]:
mini_batch_excess_padding_len = mini_batch_dict["logprobs"].shape[1] - logprobs_new.shape[1]


aligned_mini_batch_dict = {
    "logprobs": mini_batch_dict["logprobs"][:, mini_batch_excess_padding_len:],
    "values": mini_batch_dict["values"][:, mini_batch_excess_padding_len:],
    "masks": mini_batch_dict["masks"][:, mini_batch_excess_padding_len:],
}

In [9]:
from trl.core import (
    PPODecorators,
    logprobs_from_logits,
    WANDB_PADDING,
    stack_dicts,
    stats_to_np,
    convert_to_scalar,
)

logprobs_from_logits

<function trl.core.logprobs_from_logits(logits, labels, gather=True)>

In [10]:
import math

In [14]:
for i in range(math.ceil(bs / fbs)):
    input_kwargs = {key: value[i * fbs : (i + 1) * fbs] for key, value in model_inputs.items()}
    query_batch = queries[i * fbs : (i + 1) * fbs]
    response_batch = responses[i * fbs : (i + 1) * fbs]
    with torch.no_grad():
        logits, _, values = model(**input_kwargs)

    input_ids = input_kwargs["input_ids"]
    attention_mask = input_kwargs["attention_mask"]

    _, attention_mask, image_indicator_mask, input_ids = (
        training.get_llava_image_embedding_index_range_for_multimodal_batch_for_ppo(
            model=model,
            input_ids=input_ids,
            attention_mask=attention_mask,
            images=input_kwargs["images"],
        )
    )  # N

    logprobs = logprobs_from_logits(logits[:, :-1, :], input_ids[:, 1:])
    masks = torch.zeros_like(attention_mask)
    masks[:, :-1] = attention_mask[:, 1:]

    for j in range(len(query_batch)):
        num_image_embedding_tokens_in_query = (
            image_indicator_mask[j].nonzero(as_tuple=True)[0].shape[0]
        )
        start = len(query_batch[j]) + num_image_embedding_tokens_in_query - 1
        if attention_mask[j, 0] == 0:  # offset left padding
            start += attention_mask[j, :].nonzero()[0]
        end = start + len(response_batch[j])

        masks[j, :start] = 0
        masks[j, end:] = 0

    return_logits = True
    if return_logits:
        all_logits.append(logits)
    else:
        del logits
    all_values.append(values)
    all_logprobs.append(logprobs)
    all_masks.append(masks)

In [21]:
all_logits[0].requires_grad

False

In [ ]:
import torch
import torch.nn.functional as F


padded_logits, padded_values, padded_logprobs, padded_masks = pad_mini_batches(
    all_logits=all_logits,
    all_values=all_values,
    all_logprobs=all_logprobs,
    all_masks=all_masks,
)

In [35]:
print(all_masks[2].nonzero(as_tuple=True)[1][0])
print(padded_masks[2].nonzero(as_tuple=True)[1][0])

tensor(387, device='cuda:0')
tensor(387, device='cuda:0')


In [17]:
for i in range(len(padded_logits)):
    print(all_logits[i].shape)
    print(all_values[i].shape)
    print(all_logprobs[i].shape)
    print(all_masks[i].shape)
    print("===" * 10)
    print(padded_logits[i].shape)
    print(padded_values[i].shape)
    print(padded_logprobs[i].shape)
    print(padded_masks[i].shape)
    print("===" * 10)

torch.Size([4, 396, 32000])
torch.Size([4, 396])
torch.Size([4, 395])
torch.Size([4, 396])
torch.Size([4, 404, 32000])
torch.Size([4, 404])
torch.Size([4, 403])
torch.Size([4, 404])
torch.Size([4, 396, 32000])
torch.Size([4, 396])
torch.Size([4, 395])
torch.Size([4, 396])
torch.Size([4, 404, 32000])
torch.Size([4, 404])
torch.Size([4, 403])
torch.Size([4, 404])
torch.Size([4, 404, 32000])
torch.Size([4, 404])
torch.Size([4, 403])
torch.Size([4, 404])
torch.Size([4, 404, 32000])
torch.Size([4, 404])
torch.Size([4, 403])
torch.Size([4, 404])
torch.Size([4, 396, 32000])
torch.Size([4, 396])
torch.Size([4, 395])
torch.Size([4, 396])
torch.Size([4, 404, 32000])
torch.Size([4, 404])
torch.Size([4, 403])
torch.Size([4, 404])


In [ ]:
all_logprobs = (torch.cat(all_logprobs),)
all_logits = (torch.cat(all_logits)[:, :-1] if return_logits else None,)
all_values = (torch.cat(all_values)[:, :-1],)
all_masks = (torch.cat(all_masks)[:, :-1],)

In [74]:
all_logprobs[0].shape

torch.Size([4, 395])

In [56]:
final_input_embeds.shape

torch.Size([4, 396, 4096])

196

In [26]:
final_input_embeds, final_attention_mask, final_image_indicator_mask, final_expanded_input_ids = (
    training.get_llava_image_embedding_index_range_for_multimodal_batch_for_ppo(
        model=model,
        input_ids=input_ids_mini,
        attention_mask=attention_mask_mini,
        images=images_mini,
    )
)

In [ ]:
logprobs = logprobs_from_logits(logits_mini[:, :-1, :], final_expanded_input_ids[:, 1:])

In [51]:
final_expanded_input_ids[0, 1:]

tensor([    0,     1,   319, 13563,  1546,   263, 12758,  1404,   322,   385,
        23116, 21082, 20255, 16684,   408,   385, 18860, 17937, 19915, 29889,
          450, 20255,  4076, 10257, 29892, 13173, 29892,   322,  1248,   568,
         6089,   304,   278,  1404, 29915, 29879,  5155, 29889,  3148,  1001,
        29901, 29871,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [50]:
logprobs[0]

tensor([-3.3248e+01,  0.0000e+00, -1.1925e+00, -5.7696e-05, -2.2170e-04,
        -4.5537e-05, -7.0331e-05, -2.9357e-04, -1.1921e-06, -3.2186e-06,
        -4.3272e-05, -3.5566e-04, -5.1020e-05, -3.8861e-05,  0.0000e+00,
        -1.1921e-07, -4.8040e-05, -1.4424e-05, -5.1260e-06, -8.1062e-06,
        -9.2486e-02, -6.7589e-05, -1.3231e-04, -2.0146e-05,  0.0000e+00,
        -1.0490e-05, -2.3842e-07, -3.5763e-07, -4.1722e-05, -2.5391e-05,
        -2.6226e-05, -9.5367e-07, -2.0266e-06, -1.2981e-04, -1.4305e-06,
        -8.6542e-05, -2.1396e-04, -9.1791e-06, -7.2477e-05, -2.3842e-07,
        -1.0729e-06, -1.5651e-04, -1.8474e+01, -1.6978e+01, -1.5011e+01,
        -2.0124e+01, -2.1078e+01, -2.0337e+01, -1.6837e+01, -1.5375e+01,
        -1.5646e+01, -1.5276e+01, -1.4638e+01, -1.4967e+01, -1.4595e+01,
        -1.4021e+01, -1.8484e+01, -1.8104e+01, -1.5261e+01, -1.5270e+01,
        -1.4162e+01, -1.3253e+01, -1.3473e+01, -1.3676e+01, -1.3490e+01,
        -1.2890e+01, -1.3422e+01, -1.3400e+01, -1.3

In [34]:
tokenizer.batch_decode(final_expanded_input_ids)

['<unk><unk><s> A chat between a curious user and an artificial intelligence assistant acting as an experienced radiologist. The assistant gives professional, detailed, and polite answers to the user\'s questions. USER: <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>

In [25]:
input_ids_mini.shape[1]

209

In [23]:
true_indices.shape[0]

200

In [21]:
true_indices = (~final_image_indicator_mask)[0].nonzero(as_tuple=True)[0]

In [ ]:
# expanded_input_ids_mini = torch.full_like(
#     final_image_indicator_mask, fill_value=-200, dtype=input_ids_mini.dtype, device=input_ids_mini.device
# )

# Process each batch row
for i in range(final_image_indicator_mask.shape[0]):
    true_indices = final_image_indicator_mask[i].nonzero(as_tuple=True)[0]
    expanded_input_ids_mini[i, true_indices] = input_ids_mini[i]

In [42]:
logits_mini.shape

torch.Size([4, 396, 32000])

In [ ]:
tokens_mini = training.get_likeliest_token_from_logits(logits_mini[:, :-1, :])
for i, tokens_to_decode in enumerate(tokens_mini):

    decoded_text = tokenizer.decode(
        tokens_to_decode[(final_attention_mask[i] & ~final_image_indicator_mask[i])[:-1]]
    )
    print("".join(decoded_text))

 A chat between a curious user and an artificial intelligence assistant acting as an experienced radiologist. The assistant gives professional, detailed, and polite answers to the user's questions. USER: 1 You are to act as a radiologist and answer a single question. After you respond, please provide your self evaluation of your confidence. Provide a confidence between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, of how sure you are the answer is correct. A value close to 0 means you think there is a high probability that the answer is wrong. Your confidence is to be reported in a JSON dictionary of the following format: {"confidence": int}. Is there any indic of  Ptelectasis in the report? ASSISTANT: No, the imageagnostic markers indicate Atelectasis. {"confidence": 1}</s>
 A chat between a curious user and an artificial intelligence assistant acting as an experienced radiologist. The assistant gives professional, detailed, and polite answers to the user's questions. USER: 1 You are to act as a 

# Archived

## Archived attempt to account for image embeddings

In [ ]:
def account_for_image_embeddings_for_single_image_inputs(input_ids, logits, values, attention_mask):
    """
    Args:
        input_ids (torch.Tensor): A tensor shaped (batch_size, sequence_length) including exactly 1 image token id (-200 for llava) for each sequence
        logits (torch.Tensor): A tensor shaped (batch_size, sequence_length, vocab_size)
        values: The values of the model
        attention_mask: The attention mask of the model
    """
    # Locate the image index
    indexes_of_image_token = (input_ids == training.LLAVA_IMAGE_TOKEN_INDEX).nonzero(as_tuple=True)[
        1
    ]

    # Shift the indexes taking in the account where each sequence begins (bos_token is taken as basis)
    indexes_of_bos_token = (input_ids == 1).int().argmax(dim=1)
    indexes_of_image_token += indexes_of_bos_token

In [ ]:
# queries = training.replace_image_token_with_another_token_for_list_of_tensors(input_ids_list)
# queries = t.cast(list[torch.LongTensor], queries)
model.gradient_checkpointing_enable()
stats = ppo_trainer.multimodal_step(
    queries=t.cast(list[torch.LongTensor], input_ids_list),
    responses=t.cast(list[torch.LongTensor], generated_confidences_ids),
    scores=rewards,
    images=images,
)

In [17]:
image_features = model.pretrained_model.base_model.model.model.get_vision_tower()(
    images
).patch_embeddings
image_features = image_features.flatten(2).transpose(1, 2)
image_features = model.pretrained_model.base_model.model.model.mm_projector(image_features)
image_features.shape

torch.Size([16, 196, 4096])

In [48]:
input_ids, position_ids, attention_mask, past_key_values, inputs_embeds, labels = (
    model.pretrained_model.base_model.model.prepare_inputs_labels_for_multimodal(
        input_ids=input_ids,
        position_ids=None,
        attention_mask=attention_mask,
        past_key_values=None,
        labels=None,
        images=images,
    )
)

## Prompting Experiments

In [17]:
POST_GENERATION_CONFIDENCE_REQUEST_1 = (
    "USER: Now evaluate your own response. How confident are you in your answer? "
    "Provide a confidence between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, of how sure you are the answer is correct. "
    "A value close to 0 means you think there is a high probability that the answer is wrong. "
    "The closer the value is to 10, the higher you think is the probability that the answer is correct. "
    "The output should have the format 'Confidence: <confidence>' and nothing else. "
)


POST_GENERATION_CONFIDENCE_REQUEST_2 = (
    "USER: Now evaluate your own response. How confident are you in your answer? "
    "Provide a confidence between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, of how sure you are the answer is correct. "
    "The output should have the format 'My Confidence: <confidence>'. "
    "ASSISTANT: Here is my self evaluation of my confidence. My Confidence (out of 10):"
)

POST_GENERATION_CONFIDENCE_REQUEST_3 = "USER: Irrelevant from my question, what is your name?"

POST_GENERATION_CONFIDENCE_REQUEST_4 = (
    "USER: Now evaluate your own response. How confident are you in your answer? "
    "Provide a confidence between 0, 1, 2, 3, 4, 5 of how sure you are the answer is correct. "
    "The output should have the format 'My Confidence: <confidence>' My Reasoning for my Self Confidence Evaluation: <reasoning>. "
    "ASSISTANT: Here is my self evaluation of my confidence. My Confidence (out of 5):"
)

POST_GENERATION_CONFIDENCE_REQUEST_5 = (
    "USER: Now evaluate your own response. How confident are you in your answer? "
    "Provide a confidence between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,  of how sure you are the answer is correct. "
    "The output should have the format 'My Confidence: <confidence>' My Very Short Reasoning for my Self Confidence Evaluation: <very_short_reasoning>. "
    "ASSISTANT: Here is my self evaluation of my confidence. My Confidence (out of 10):"
)

POST_GENERATION_CONFIDENCE_REQUEST_6 = (
    "USER: Now evaluate your own response. How confident are you in your answer and what is your explanation regarding your confidence?"
    "ASSISTANT: When asked how confident I am about a response, I consistently provide it in a JSON object, adhering to my policy."
    "My confidence is a value between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, where 0 means I am not confident at all and 10 means I am very confident."
    "The confidence JSON follows this structure: {'confidence': int, 'explanation': str}."
    "Here's my confidence JSON about my last response: "
)

POST_GENERATION_CONFIDENCE_REQUEST_7 = (
    "USER: Now evaluate your own response. How confident are you in your answer?"
    "ASSISTANT: When asked how confident I am about a response, I consistently provide it in a JSON object, adhering to my policy."
    "My confidence is a value between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, where 0 means I am not confident at all and 10 means I am very confident."
    "The confidence JSON follows this structure: {'confidence': int}."
    "Here's my confidence JSON about my last response: "
)

POST_GENERATION_CONFIDENCE_REQUEST_8 = (
    "</s> USER: Now evaluate your own response. How confident are you in your answer?"
    "Provide a confidence between 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, of how sure you are the answer is correct. "
    "A value close to 0 means you think there is a high probability that the answer is wrong. "
    "The closer the value is to 10, the higher you think is the probability that the answer is correct. "
    "ASSISTANT: When asked how confident I am about a response, I consistently provide it in a JSON object, adhering to my policy."
    "The confidence JSON follows this structure: {'confidence': int}."
    "Here's my confidence JSON about my last response: "
)

In [5]:
tokenizer = vllm.load_pretrained_llava_tokenizer_with_image_support(
    model_base=vllm.LLAVA_BASE_MODEL_NAME
)

In [ ]:
STOP_STR = prompter.Seperator.END_OF_SEQUENCE_SEPERATOR.value
from LLAVA_Biovil.llava.mm_utils import KeywordsStoppingCriteria
from RewardingVisualDoubt import inference

padding_tokenizer = vllm.load_pretrained_llava_tokenizer_with_image_support(
    model_base=vllm.LLAVA_BASE_MODEL_NAME
)
padding_tokenizer.padding_side = "left"
padding_tokenizer.pad_token_id = padding_tokenizer.bos_token_id
dataset_test = dataset.get_binary_qa_prompted_mimic_cxr_llava_model_input_dataset(
    split=dataset.DatasetSplit.TEST,
    tokenizer=tokenizer,
    prompter=prompter.build_binary_qa_instruction_from_disease_under_study,
)
dataloader_test = dataset.get_mimic_cxr_llava_model_input_dataloader(
    dataset=dataset_test, batch_size=1, padding_tokenizer=padding_tokenizer, num_workers=8
)

for idx, batch in enumerate(dataloader_test):
    batch = t.cast(dataset.MimicCxrLlavaModelInputBatchDict, batch)
    batch_llava_model_input_dict = batch["batch_llava_model_input_dict"]
    batch_llava_model_input_dict = dataset.move_llava_model_input_dict_to_device(
        batch_llava_model_input_dict, torch.device(shared.torch_devices.cuda.value)
    )
    input_ids, images = (
        batch_llava_model_input_dict["text_prompt_input_ids"],
        batch_llava_model_input_dict["images"],
    )
    stopping_criteria = KeywordsStoppingCriteria([STOP_STR], tokenizer, input_ids)
    pred = inference.generate_radialog_answer_for_binary_qa_for_single_study(
        model, tokenizer, input_ids, images, stopping_criteria
    )
    confidence_request_prompt = (
        batch["batch_prompts"][0]
        + " "
        + pred
        + " "
        + prompter.build_post_generation_user_confidence_request()  # POST_GENERATION_CONFIDENCE_REQUEST_8
    )
    confidence_request_input_ids = torch.unsqueeze(
        torch.IntTensor(tokenizer(confidence_request_prompt)["input_ids"]), 0
    ).to(device)
    stopping_criteria = KeywordsStoppingCriteria(
        [STOP_STR], tokenizer, confidence_request_input_ids
    )
    pred_with_confidence = inference.generate_radialog_answer_for_binary_qa_for_single_study(
        model, tokenizer, confidence_request_input_ids, images, stopping_criteria
    )
    print(f"\n Metadata: {batch['batch_mimic_cxr_datapoint_metadata']}")
    print(f"Prompt: {batch['batch_prompts']}")
    print(f"Label:", batch["batch_labels"])
    print(f"File_idx {idx}, ASSISTANT: ", pred)
    print(f"File_idx {idx}, ASSISTANT (after confidence request): ", pred_with_confidence)
    if idx == 5:
        break

In [ ]:
######################################## TEST TO SEE IF TEMPERATURE AND TOP_P PARAMS HELP WITH USER CONFIDENCE REQUEST WITHOUT ASSISTANT CONFIRMATION ########################################


from LLAVA_Biovil.llava.mm_utils import tokenizer_image_token

iterator_train = iter(dataloader_train)
batch = next(iterator_train)

batch_llava_model_input_dict = batch["batch_llava_model_input_dict"]
batch_llava_model_input_dict = dataset.move_llava_model_input_dict_to_device(
    batch_llava_model_input_dict, device
)
_, images = (
    batch_llava_model_input_dict["text_prompt_input_ids"],
    batch_llava_model_input_dict["images"],
)

my_prompt = prompter.build_binary_qa_instruction_from_disease_under_study_with_confidence_request(
    "Cardiomegaly"
)
tokenized_prompt = tokenizer_image_token(my_prompt, tokenizer, return_tensors="pt").to(device)

stopping_criteria = KeywordsStoppingCriteria([STOP_STR], tokenizer, tokenized_prompt.unsqueeze(0))

prompt_and_generated_answers_ids = model.generate(
    input_ids=tokenized_prompt.unsqueeze(0),
    images=images[0].unsqueeze(0),
    # attention_mask=attention_mask,
    do_sample=True,
    use_cache=True,
    temperature=1.8,
    top_p=0.7,
    max_new_tokens=300,  # TODO maybe move to the kwargs
    stopping_criteria=[stopping_criteria],  # TODO understand better
    pad_token_id=tokenizer.pad_token_id,  # used in tokenizing after the generation, # TODO maybe move to the kwargs
    # **generation_kwargs_prediction,  # TODO check which args to pass.
)

tokenizer.decode(
    training.replace_image_token_with_another_token(prompt_and_generated_answers_ids)[0]
)

## Archived

In [ ]:
batch = next(iterator_train)

batch = t.cast(dataset.MimicCxrLlavaModelInputBatchDict, batch)
batch_llava_model_input_dict = batch["batch_llava_model_input_dict"]
batch_llava_model_input_dict = dataset.move_llava_model_input_dict_to_device(
    batch_llava_model_input_dict, device
)
input_ids, images = (
    batch_llava_model_input_dict["text_prompt_input_ids"],
    batch_llava_model_input_dict["images"],
)
attention_mask = batch["batch_attention_mask"].to(device)  # TODO handle elsewhere
labels = batch["batch_labels"].to(device)  # TODO handle elsewhere


model.eval()
stopping_criteria = KeywordsStoppingCriteria([STOP_STR], tokenizer, input_ids)


t3 = time.time()
prompt_and_generated_answers_ids = model.generate(
    input_ids=input_ids,
    images=images,
    attention_mask=attention_mask,
    do_sample=False,
    use_cache=True,
    max_new_tokens=32,  # Limiting, YES, but binary q&a answers are not very long!
    stopping_criteria=[stopping_criteria],
    pad_token_id=tokenizer.pad_token_id,
)
t4 = time.time()
prompt_and_generated_answers_ids = training.remove_trailing_padding_from_prediction(
    prompt_and_generated_answers_ids, tokenizer.pad_token_id
)

# Append confidence request to the generated answers
prompt_and_generated_answers_with_confidence_requests_ids = []
for item in prompt_and_generated_answers_ids:
    confidence_request_input_ids = (
        tokenizer(prompter.build_post_generation_user_confidence_request(), return_tensors="pt")
        .input_ids.to(device)
        .squeeze(0)
    )[
        1:
    ]  # drop start of sequence token
    prompt_and_generated_answers_with_confidence_requests_ids.append(
        torch.cat((item, confidence_request_input_ids), 0)
    )
model.train()

t5 = time.time()
generated_confidences_ids = ppo_trainer.generate(
    prompt_and_generated_answers_with_confidence_requests_ids,  # ppo_trainer.generate() method admits list of tensors, not a batch tensor unfortunately
    images=images,
    return_prompt=False,
    **generation_kwargs_ppo,
)
t6 = time.time()


complete_conversation_ids = [
    torch.cat((p, c), 0)
    for p, c in zip(
        prompt_and_generated_answers_with_confidence_requests_ids,
        generated_confidences_ids,
    )
]
generated_answer_only_ids = [
    prompt_and_generated_answers_ids[i][len(input_ids[i]) :] for i in range(len(input_ids))
]

# Remove the unindex image token from the prompt
prompt_and_generated_answers_with_confidence_requests_ids = (
    training.replace_image_token_with_another_token_for_list_of_tensors(
        prompt_and_generated_answers_with_confidence_requests_ids
    )
)
generated_answers_texts = tokenizer.batch_decode(
    generated_answer_only_ids,
    skip_special_tokens=True,
)
generated_confidences_texts = tokenizer.batch_decode(
    generated_confidences_ids,
    skip_special_tokens=True,
)
generated_answer_labels = response.parse_binary_labels(generated_answers_texts)
generated_confidence_values = response.parse_confidences(generated_confidences_texts)

rewards = [
    reward.generated_answer_and_confidence_to_reward(
        generated_answer_label, generated_confidence_value, ground_truth_label
    )
    for generated_answer_label, generated_confidence_value, ground_truth_label in zip(
        generated_answer_labels, generated_confidence_values, labels.bool().tolist()
    )
]

report = {}
report["generated_answer_labels"] = generated_answer_labels

rewards_epoch += rewards
rewards = [torch.tensor(r).to(device) for r in rewards]

t7 = time.time()
stats = ppo_trainer.step(
    prompt_and_generated_answers_with_confidence_requests_ids, generated_answer_only_ids, rewards
)
t8 = time.time()

# ppo_trainer.log_stats(stats, batch, rewards, columns_to_log=["query", "response", "answer"])

# print(f"Finished epoch {epoch}. Average reward: {avg_reward}")
# ppo_trainer.save_pretrained(os.path.join(out_dir, "model_finetuned"))

# TODO: For random exploration
# chance_to_change_confidence -= reduce_per_step
# chance_to_change_confidence = max(0, chance_to_change_confidence)

In [ ]:
working_set = "mini_batch"
input_ids_idx = 1


if working_set == "mini_batch":
    print("Working with mini batch")
    input_ids_working = input_ids_mini.clone().detach()
    logits_working = logits_mini.clone().detach()
elif working_set == "full_batch":
    print("Working with full batch")
    input_ids_working = input_ids.clone().detach()
    logits_working = logits.clone().detach()
else:
    raise ValueError(
        f"working_set must be one of ['mini_batch', 'full_batch'], but got {working_set}"
    )


indexes_of_image_token = (input_ids_working == training.LLAVA_IMAGE_TOKEN_INDEX).nonzero(
    as_tuple=True
)[1]

print(
    tokenizer.batch_decode(
        torch.argmax(logits_working[input_ids_idx][: indexes_of_image_token[input_ids_idx]], dim=-1)
    )
)
print(
    tokenizer.batch_decode(
        torch.argmax(
            logits_working[input_ids_idx][
                indexes_of_image_token[input_ids_idx] : indexes_of_image_token[input_ids_idx] + 196
            ],
            dim=-1,
        )
    )
)
print(
    tokenizer.batch_decode(
        torch.argmax(
            logits_working[input_ids_idx][indexes_of_image_token[input_ids_idx] + 196 :], dim=-1
        )
    )
)
print("\n ")

print(input_ids_working[input_ids_idx])

print("\n \n ")

Working with mini batch
['<s>', '<s>', 'A', 'chat', 'between', 'a', 'curious', 'user', 'and', 'an', 'artificial', 'intelligence', 'assistant', 'acting', 'as', 'an', 'experienced', 'radi', 'ologist', '.', 'The', 'assistant', 'gives', 'professional', ',', 'detailed', ',', 'and', 'pol', 'ite', 'answers', 'to', 'the', 'user', "'", 's', 'questions', '.', 'US', 'ER', ':', '', '1', '', '.', 'ã', 'is', '', ',', ',', ',']
[',', '.', '.', '.', '.', '.', '.', '.', ',', 'central', ':', ':', ':', '.', '.', '.', '.', '.', '.', '.', '.', '>', ':', '"', 'is', ',', '.', '', '.', '.', '.', '.', '', '`', '.', 'is', ':', ':', 'is', ',', '.', '', '', '.', '.', '.', '\xa0', '.', '.', '\xa0', ':', ':', 'SS', 'VICE', 'IE', 'The', ':', ':', '"', '"', '`', '.', '.', 'I', 'The', 'IE', 'URE', 'ICE', 'IE', 'IE', 'IE', 'The', '"', 'I', '.', '.', '**', '.', 'The', 'IE', 'IE', 'IE', 'IE', 'IE', 'IE', 'The', '"', '.', '.', '.', '"', 'SS', 'lung', '"', 'The', 'IE', '.', 'The', 'The', 'The', '.', '.', '.', '.', '.', '.'